In [2]:
import os
import pickle
import datetime
import glob
import numpy as np
from collections import defaultdict
from tqdm import tqdm
import logging

# For NLP processing and modeling
import gensim
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser

# Configure logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

class RedditWord2VecBuilder:
    def __init__(self, 
                 base_data_dir="processed_comments",
                 model_dir="models/model_v5",
                 vector_size=300,
                 window=5,
                 min_count=5,
                 epochs=5,
                 workers=16,
                 sg=1,  # Skip-gram (1) vs CBOW (0)
                 min_comments_to_train=100000):
        """
        Initialize the model builder with parameters
        
        Args:
            base_data_dir: Directory containing processed comment pickle files
            model_dir: Directory to save trained models
            vector_size: Dimension of word vectors
            window: Maximum distance between current and predicted word
            min_count: Minimum word count for inclusion in vocabulary
            epochs: Number of training epochs
            workers: Number of worker threads
            sg: Training algorithm: 1 for skip-gram, 0 for CBOW
            min_comments_to_train: Minimum number of comments required for training
        """
        self.base_data_dir = base_data_dir
        self.model_dir = model_dir
        self.vector_size = vector_size
        self.window = window
        self.min_count = min_count
        self.epochs = epochs
        self.workers = workers
        self.sg = sg
        self.min_comments_to_train = min_comments_to_train
        
        # Ensure model directories exist
        os.makedirs(f"{self.model_dir}/interim", exist_ok=True)
        
        # Time periods for analysis
        self.time_periods = ["before_2016", "2017_2020", "2021_2024"]
        
        # Initialize models for each time period
        self.models = {period: None for period in self.time_periods}
        self.bigram_models = {period: None for period in self.time_periods}
        
    def _get_date_from_comment(self, comment):
        """Extract date from a comment dictionary"""
        try:
            return datetime.datetime.strptime(comment["date"], "%Y-%m-%d").date()
        except (KeyError, ValueError):
            # If date cannot be parsed, try using timestamp
            try:
                return datetime.datetime.fromtimestamp(int(comment["timestamp"])).date()
            except (KeyError, ValueError):
                return None
    
    def _get_period(self, date):
        """Determine which time period a date belongs to"""
        if date is None:
            return None
            
        year = date.year
        if year <= 2016:
            return "before_2016"
        elif 2017 <= year <= 2020:
            return "2017_2020"
        elif 2021 <= year <= 2024:
            return "2021_2024"
        return None
    
    def _build_bigram_model(self, comments, period, min_count=5):
        """Build a bigram model for the given comments"""
        print(f"Building bigram model for {period} with {len(comments)} comments...")
        
        # Extract just the processed text from comments
        sentences = [comment["processed_text"] for comment in comments if "processed_text" in comment]
        
        # Create or update bigram model
        if self.bigram_models[period] is None:
            # First time creation
            phrases = Phrases(sentences, min_count=min_count, threshold=0.7, scoring='npmi')
            self.bigram_models[period] = Phraser(phrases)
        else:
            # For updates, we need to create a new Phrases model, train it on all sentences,
            # and then create a new Phraser
            print(f"Creating new bigram model for {period} (cannot update Phraser objects)")
            phrases = Phrases(sentences, min_count=min_count, threshold=0.7, scoring='npmi')
            self.bigram_models[period] = Phraser(phrases)
            
        return self.bigram_models[period]
    
    def _apply_bigrams(self, comments, bigram_model):
        """Apply bigram model to comments"""
        processed_comments = []
        
        for comment in comments:
            if "processed_text" in comment:
                # Apply bigram transformation to processed text
                bigrammed_text = bigram_model[comment["processed_text"]]
                processed_comments.append(bigrammed_text)
                
        return processed_comments
    
    def _create_or_update_model(self, comments, period):
        """Create a new model or update an existing one"""
        if self.models[period] is None:
            # Create new model
            print(f"Creating new Word2Vec model for {period}")
            model = Word2Vec(
                vector_size=self.vector_size,
                window=self.window,
                min_count=self.min_count,
                workers=self.workers,
                sg=self.sg
            )
            model.build_vocab(comments)
        else:
            # Update existing model
            print(f"Updating existing Word2Vec model for {period}")
            model = self.models[period]
            model.build_vocab(comments, update=True)
            
        # Train model
        print(f"Training model on {len(comments)} comments")
        model.train(
            comments,
            total_examples=len(comments),
            epochs=self.epochs
        )
        
        self.models[period] = model
        return model
    
    def _save_model(self, model, subreddit, period, is_interim=False):
        """Save model to disk"""
        if is_interim:
            path = f"{self.model_dir}/interim/{subreddit}_{period}_interim.model"
        else:
            path = f"{self.model_dir}/{subreddit}_{period}.model"
            
        model.save(path)
        print(f"Model saved to {path}")
    
    def find_pickle_files(self, subreddit):
        """Find all pickle files for a subreddit"""
        pattern = f"{self.base_data_dir}/{subreddit}/{subreddit}_batch*.pkl"
        files = sorted(glob.glob(pattern))
        if not files:
            print(f"No pickle files found for {subreddit} in {self.base_data_dir}/{subreddit}/")
        return files
    
    def process_pickle_file(self, file_path):
        """Load and process a pickle file"""
        try:
            with open(file_path, 'rb') as f:
                comments = pickle.load(f)
            print(f"Loaded {len(comments)} comments from {file_path}")
            return comments
        except Exception as e:
            print(f"Error loading {file_path}: {e}")
            return []
    
    def build_models(self, subreddit, chunk_size=1000000):
        """
        Build Word2Vec models for a subreddit by time period
        
        Args:
            subreddit: Name of the subreddit to process
            chunk_size: Number of comments to process in each chunk
        """
        print(f"Building models for {subreddit}")
        
        # Find all pickle files
        pickle_files = self.find_pickle_files(subreddit)
        if not pickle_files:
            return
        
        # Process each file
        comments_by_period = {period: [] for period in self.time_periods}
        
        for file_path in pickle_files:
            print(f"Processing {file_path}")
            comments = self.process_pickle_file(file_path)
            
            # Sort comments by date if not already sorted
            for comment in comments:
                date = self._get_date_from_comment(comment)
                period = self._get_period(date)
                
                if period:
                    comments_by_period[period].append(comment)
            
            # Process each time period separately
            for period in self.time_periods:
                period_comments = comments_by_period[period]
                
                # If we have enough comments, train in chunks
                while len(period_comments) >= chunk_size:
                    print(f"Processing chunk of {chunk_size} comments for {period}")
                    
                    # Take a chunk of comments
                    chunk = period_comments[:chunk_size]
                    period_comments = period_comments[chunk_size:]
                    
                    # Build or update bigram model
                    bigram_model = self._build_bigram_model(chunk, period)
                    
                    # Apply bigrams
                    processed_chunk = self._apply_bigrams(chunk, bigram_model)
                    
                    if len(processed_chunk) > self.min_comments_to_train:
                        # Create or update model
                        model = self._create_or_update_model(processed_chunk, period)
                        
                        # Save interim model
                        self._save_model(model, subreddit, period, is_interim=True)
                
                # Store remaining comments for next file
                comments_by_period[period] = period_comments
        
        # Process any remaining comments that didn't make a full chunk
        for period, remaining_comments in comments_by_period.items():
            if len(remaining_comments) > self.min_comments_to_train:
                print(f"Processing final {len(remaining_comments)} comments for {period}")
                
                # Build or update bigram model
                bigram_model = self._build_bigram_model(remaining_comments, period)
                
                # Apply bigrams
                processed_chunk = self._apply_bigrams(remaining_comments, bigram_model)
                
                # Create or update model
                model = self._create_or_update_model(processed_chunk, period)
                
                # Save final model
                if model is not None:
                    self._save_model(model, subreddit, period, is_interim=False)
            else:
                print(f"Skipping final {len(remaining_comments)} comments for {period} (less than minimum required)")
        
        # Save final models
        for period, model in self.models.items():
            if model is not None:
                self._save_model(model, subreddit, period, is_interim=False)
        
        print(f"Completed building models for {subreddit}")


def main():
    # Initialize the model builder
    builder = RedditWord2VecBuilder(
        base_data_dir="processed_comments",
        model_dir="models/model_v5",
        vector_size=300,
        window=5,
        min_count=10,  # Minimum word frequency
        epochs=5,
        workers=16,
        sg=1,  # Skip-gram model
        min_comments_to_train=100000
    )
    
    # List of subreddits to process
    subreddits = ["democrats", "republican", "conservative", "liberal", "backpacking", "vagabond"]
    
    # Build models for each subreddit
    for subreddit in subreddits:
        builder.build_models(subreddit, chunk_size=1000000)


if __name__ == "__main__":
    main()

Building models for democrats
Processing processed_comments/democrats\democrats_batch1.pkl
Loaded 1000000 comments from processed_comments/democrats\democrats_batch1.pkl
Processing processed_comments/democrats\democrats_batch2.pkl
Loaded 935437 comments from processed_comments/democrats\democrats_batch2.pkl


2025-08-19 17:59:36,820 : INFO : collecting all words and their counts
2025-08-19 17:59:36,821 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2025-08-19 17:59:36,848 : INFO : PROGRESS: at sentence #10000, processed 75608 words and 62464 word types
2025-08-19 17:59:36,888 : INFO : PROGRESS: at sentence #20000, processed 180000 words and 135361 word types


Processing chunk of 1000000 comments for 2021_2024
Building bigram model for 2021_2024 with 1000000 comments...


2025-08-19 17:59:36,934 : INFO : PROGRESS: at sentence #30000, processed 298263 words and 207549 word types
2025-08-19 17:59:36,976 : INFO : PROGRESS: at sentence #40000, processed 404663 words and 267973 word types
2025-08-19 17:59:37,027 : INFO : PROGRESS: at sentence #50000, processed 531060 words and 335115 word types
2025-08-19 17:59:37,076 : INFO : PROGRESS: at sentence #60000, processed 648297 words and 391050 word types
2025-08-19 17:59:37,125 : INFO : PROGRESS: at sentence #70000, processed 775037 words and 452188 word types
2025-08-19 17:59:37,171 : INFO : PROGRESS: at sentence #80000, processed 889776 words and 505802 word types
2025-08-19 17:59:37,220 : INFO : PROGRESS: at sentence #90000, processed 1010201 words and 559857 word types
2025-08-19 17:59:37,270 : INFO : PROGRESS: at sentence #100000, processed 1129841 words and 610802 word types
2025-08-19 17:59:37,320 : INFO : PROGRESS: at sentence #110000, processed 1249037 words and 662914 word types
2025-08-19 17:59:37,381

Creating new Word2Vec model for 2021_2024


...

Training model on 292174 comments


2025-08-19 18:15:39,847 : INFO : EPOCH 0 - PROGRESS: at 68.27% examples, 2800711 words/s, in_qsize 31, out_qsize 0
2025-08-19 18:15:40,219 : INFO : EPOCH 0: training on 4374540 raw words (4025870 effective words) took 1.4s, 2930474 effective words/s
2025-08-19 18:15:41,234 : INFO : EPOCH 1 - PROGRESS: at 70.97% examples, 2909227 words/s, in_qsize 32, out_qsize 0
2025-08-19 18:15:41,598 : INFO : EPOCH 1: training on 4374540 raw words (4026024 effective words) took 1.4s, 2937452 effective words/s
2025-08-19 18:15:42,608 : INFO : EPOCH 2 - PROGRESS: at 73.56% examples, 3034292 words/s, in_qsize 31, out_qsize 0
2025-08-19 18:15:42,899 : INFO : EPOCH 2: training on 4374540 raw words (4026590 effective words) took 1.3s, 3115963 effective words/s
2025-08-19 18:15:43,911 : INFO : EPOCH 3 - PROGRESS: at 72.07% examples, 2963225 words/s, in_qsize 31, out_qsize 0
2025-08-19 18:15:44,237 : INFO : EPOCH 3: training on 4374540 raw words (4025967 effective words) took 1.3s, 3028520 effective words/s


Model saved to models/word2vec/vagabond_2021_2024.model
Model saved to models/word2vec/vagabond_before_2016.model
Model saved to models/word2vec/vagabond_2017_2020.model


2025-08-19 18:15:45,793 : INFO : saved models/word2vec/vagabond_2021_2024.model


Model saved to models/word2vec/vagabond_2021_2024.model
Completed building models for vagabond


In [ ]:
import os
import pickle
import datetime
import glob
import numpy as np
from collections import defaultdict
from tqdm import tqdm
import logging

# For NLP processing and modeling
import gensim
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser

# Configure logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

class RedditWord2VecBuilder:
    def __init__(self, 
                 base_data_dir="processed_comments",
                 model_dir="models/yearly_models",  # Changed location
                 vector_size=300,
                 window=5,
                 min_count=5,
                 epochs=5,
                 workers=16,
                 sg=1,  # Skip-gram (1) vs CBOW (0)
                 min_comments_to_train=100000,
                 start_year=2024,
                 end_year=2024):
        """
        Initialize the model builder with parameters
        
        Args:
            base_data_dir: Directory containing processed comment pickle files
            model_dir: Directory to save trained models
            vector_size: Dimension of word vectors
            window: Maximum distance between current and predicted word
            min_count: Minimum word count for inclusion in vocabulary
            epochs: Number of training epochs
            workers: Number of worker threads
            sg: Training algorithm: 1 for skip-gram, 0 for CBOW
            min_comments_to_train: Minimum number of comments required for training
            start_year: First year to analyze
            end_year: Last year to analyze
        """
        self.base_data_dir = base_data_dir
        self.model_dir = model_dir
        self.vector_size = vector_size
        self.window = window
        self.min_count = min_count
        self.epochs = epochs
        self.workers = workers
        self.sg = sg
        self.min_comments_to_train = min_comments_to_train
        self.start_year = start_year
        self.end_year = end_year
        
        # Ensure model directories exist
        os.makedirs(f"{self.model_dir}/interim", exist_ok=True)
        
        # Generate years for analysis
        self.years = list(range(start_year, end_year + 1))
        
        # Initialize models for each year
        self.models = {str(year): None for year in self.years}
        self.bigram_models = {str(year): None for year in self.years}
        
    def _get_date_from_comment(self, comment):
        """Extract date from a comment dictionary"""
        try:
            return datetime.datetime.strptime(comment["date"], "%Y-%m-%d").date()
        except (KeyError, ValueError):
            # If date cannot be parsed, try using timestamp
            try:
                return datetime.datetime.fromtimestamp(int(comment["timestamp"])).date()
            except (KeyError, ValueError):
                return None
    
    def _get_year(self, date):
        """Extract year from date"""
        if date is None:
            return None
        
        year = date.year
        if self.start_year <= year <= self.end_year:
            return str(year)
        return None
    
    def _build_bigram_model(self, comments, year, min_count=5):
        """Build a bigram model for the given comments"""
        print(f"Building bigram model for {year} with {len(comments)} comments...")
        
        # Extract just the processed text from comments
        sentences = [comment["processed_text"] for comment in comments if "processed_text" in comment]
        
        # Create or update bigram model
        if self.bigram_models[year] is None:
            # First time creation
            phrases = Phrases(sentences, min_count=min_count, threshold=0.7, scoring='npmi')
            self.bigram_models[year] = Phraser(phrases)
        else:
            # For updates, we need to create a new Phrases model with all sentences
            print(f"Creating new bigram model for {year} (cannot update Phraser objects)")
            phrases = Phrases(sentences, min_count=min_count, threshold=0.7, scoring='npmi')
            self.bigram_models[year] = Phraser(phrases)
            
        return self.bigram_models[year]
    
    def _apply_bigrams(self, comments, bigram_model):
        """Apply bigram model to comments"""
        processed_comments = []
        
        for comment in comments:
            if "processed_text" in comment:
                # Apply bigram transformation to processed text
                bigrammed_text = bigram_model[comment["processed_text"]]
                processed_comments.append(bigrammed_text)
                
        return processed_comments
    
    def _create_or_update_model(self, comments, year):
        """Create a new model or update an existing one"""
        if self.models[year] is None:
            # Create new model
            print(f"Creating new Word2Vec model for {year}")
            model = Word2Vec(
                vector_size=self.vector_size,
                window=self.window,
                min_count=self.min_count,
                workers=self.workers,
                sg=self.sg
            )
            model.build_vocab(comments)
        else:
            # Update existing model
            print(f"Updating existing Word2Vec model for {year}")
            model = self.models[year]
            model.build_vocab(comments, update=True)
            
        # Train model
        print(f"Training model on {len(comments)} comments")
        model.train(
            comments,
            total_examples=len(comments),
            epochs=self.epochs
        )
        
        self.models[year] = model
        return model
    
    def _save_model(self, model, subreddit, year, is_interim=False):
        """Save model to disk"""
        if is_interim:
            path = f"{self.model_dir}/interim/{subreddit}_{year}_interim.model"
        else:
            path = f"{self.model_dir}/{subreddit}_{year}.model"
            
        model.save(path)
        print(f"Model saved to {path}")
    
    def find_pickle_files(self, subreddit):
        """Find all pickle files for a subreddit"""
        pattern = f"{self.base_data_dir}/{subreddit}/{subreddit}_batch*.pkl"
        files = sorted(glob.glob(pattern))
        if not files:
            print(f"No pickle files found for {subreddit} in {self.base_data_dir}/{subreddit}/")
        return files
    
    def process_pickle_file(self, file_path):
        """Load and process a pickle file"""
        try:
            with open(file_path, 'rb') as f:
                comments = pickle.load(f)
            print(f"Loaded {len(comments)} comments from {file_path}")
            return comments
        except Exception as e:
            print(f"Error loading {file_path}: {e}")
            return []
    
    def build_models(self, subreddit, chunk_size=1000000):
        """
        Build Word2Vec models for a subreddit by year
        
        Args:
            subreddit: Name of the subreddit to process
            chunk_size: Number of comments to process in each chunk
        """
        print(f"Building yearly models for {subreddit}")
        
        # Find all pickle files
        pickle_files = self.find_pickle_files(subreddit)
        if not pickle_files:
            return
        
        # Process each file
        comments_by_year = {str(year): [] for year in self.years}
        year_counts = {str(year): 0 for year in self.years}
        
        for file_path in pickle_files:
            print(f"Processing {file_path}")
            comments = self.process_pickle_file(file_path)
            
            # Sort comments by year
            for comment in comments:
                date = self._get_date_from_comment(comment)
                year = self._get_year(date)
                
                if year:
                    comments_by_year[year].append(comment)
                    year_counts[year] += 1
            
            # Process each year separately
            for year in [str(y) for y in self.years]:
                year_comments = comments_by_year[year]
                
                # If we have enough comments, train in chunks
                while len(year_comments) >= chunk_size:
                    print(f"Processing chunk of {chunk_size} comments for {year}")
                    
                    # Take a chunk of comments
                    chunk = year_comments[:chunk_size]
                    year_comments = year_comments[chunk_size:]
                    
                    # Build or update bigram model
                    bigram_model = self._build_bigram_model(chunk, year)
                    
                    # Apply bigrams
                    processed_chunk = self._apply_bigrams(chunk, bigram_model)
                    
                    if len(processed_chunk) > self.min_comments_to_train:
                        # Create or update model
                        model = self._create_or_update_model(processed_chunk, year)
                        
                        # Save interim model
                        self._save_model(model, subreddit, year, is_interim=True)
                
                # Store remaining comments for next file
                comments_by_year[year] = year_comments
        
        print("\n=== Comment Counts by Year ===")
        for year, count in year_counts.items():
            if count > 0:
                print(f"{year}: {count} comments")
        
        # Process any remaining comments that didn't make a full chunk
        for year in [str(y) for y in self.years]:
            remaining_comments = comments_by_year[year]
            if len(remaining_comments) > self.min_comments_to_train:
                print(f"Processing final {len(remaining_comments)} comments for {year}")
                
                # Build or update bigram model
                bigram_model = self._build_bigram_model(remaining_comments, year)
                
                # Apply bigrams
                processed_chunk = self._apply_bigrams(remaining_comments, bigram_model)
                
                # Create or update model
                model = self._create_or_update_model(processed_chunk, year)
                
                # Save final model
                if model is not None:
                    self._save_model(model, subreddit, year, is_interim=False)
            else:
                print(f"Skipping final {len(remaining_comments)} comments for {year} (less than minimum required)")
        
        # Save final models
        for year, model in self.models.items():
            if model is not None and year_counts[year] > 0:
                self._save_model(model, subreddit, year, is_interim=False)
        
        print(f"Completed building yearly models for {subreddit}")


def main():
    # Initialize the model builder
    builder = RedditWord2VecBuilder(
        base_data_dir="processed_comments",
        model_dir="models/yearly_models",  # Changed to yearly models directory
        vector_size=300,
        window=5,
        min_count=10,  # Minimum word frequency
        epochs=5,
        workers=16,
        sg=1,  # Skip-gram model
        min_comments_to_train=100000,
        start_year=2024,  # Adjust based on your data availability
        end_year=2024
    )
    
    # List of subreddits to process
    subreddits = ["democrats", "republican", "conservative", "liberal", "backpacking", "vagabond"]
    
    # Build models for each subreddit
    for subreddit in subreddits:
        builder.build_models(subreddit, chunk_size=1000000)


if __name__ == "__main__":
    main()

Building yearly models for democrats
Processing processed_comments/democrats\democrats_batch1.pkl
Loaded 1000000 comments from processed_comments/democrats\democrats_batch1.pkl
Processing processed_comments/democrats\democrats_batch2.pkl
Loaded 935437 comments from processed_comments/democrats\democrats_batch2.pkl


2025-08-19 19:28:41,916 : INFO : collecting all words and their counts
2025-08-19 19:28:41,916 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2025-08-19 19:28:41,979 : INFO : PROGRESS: at sentence #10000, processed 160933 words and 123961 word types
2025-08-19 19:28:42,054 : INFO : PROGRESS: at sentence #20000, processed 322626 words and 223613 word types
2025-08-19 19:28:42,116 : INFO : PROGRESS: at sentence #30000, processed 491211 words and 315058 word types



=== Comment Counts by Year ===
2011: 3420 comments
2012: 8746 comments
2013: 11045 comments
2014: 11396 comments
2015: 21153 comments
2016: 71526 comments
2017: 56790 comments
2018: 110499 comments
2019: 108852 comments
2020: 197274 comments
2021: 194381 comments
2022: 213242 comments
2023: 179248 comments
Skipping final 3420 comments for 2011 (less than minimum required)
Skipping final 8746 comments for 2012 (less than minimum required)
Skipping final 11045 comments for 2013 (less than minimum required)
Skipping final 11396 comments for 2014 (less than minimum required)
Skipping final 21153 comments for 2015 (less than minimum required)
Skipping final 71526 comments for 2016 (less than minimum required)
Skipping final 56790 comments for 2017 (less than minimum required)
Processing final 110499 comments for 2018
Building bigram model for 2018 with 110499 comments...


2025-08-19 19:28:42,193 : INFO : PROGRESS: at sentence #40000, processed 656919 words and 400681 word types
2025-08-19 19:28:42,254 : INFO : PROGRESS: at sentence #50000, processed 814707 words and 477957 word types
2025-08-19 19:28:42,329 : INFO : PROGRESS: at sentence #60000, processed 991988 words and 555034 word types
2025-08-19 19:28:42,395 : INFO : PROGRESS: at sentence #70000, processed 1149841 words and 624027 word types
2025-08-19 19:28:42,454 : INFO : PROGRESS: at sentence #80000, processed 1301045 words and 690532 word types
2025-08-19 19:28:42,534 : INFO : PROGRESS: at sentence #90000, processed 1447450 words and 750681 word types
2025-08-19 19:28:42,593 : INFO : PROGRESS: at sentence #100000, processed 1586148 words and 806963 word types
2025-08-19 19:28:42,654 : INFO : PROGRESS: at sentence #110000, processed 1735770 words and 865716 word types
2025-08-19 19:28:42,661 : INFO : collected 869482 token types (unigram + bigrams) from a corpus of 1744903 words and 110499 sente

Creating new Word2Vec model for 2018
Training model on 110499 comments


...

Model saved to models/yearly_models/backpacking_2019.model
Model saved to models/yearly_models/backpacking_2020.model
Model saved to models/yearly_models/backpacking_2021.model
Model saved to models/yearly_models/backpacking_2022.model
Model saved to models/yearly_models/backpacking_2023.model
Completed building yearly models for backpacking
Building yearly models for vagabond
Processing processed_comments/vagabond\vagabond_batch1.pkl
Loaded 488480 comments from processed_comments/vagabond\vagabond_batch1.pkl


2025-08-19 19:41:53,559 : INFO : Word2Vec lifecycle event {'fname_or_handle': 'models/yearly_models/vagabond_2013.model', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2025-08-19T19:41:53.559631', 'gensim': '4.3.3', 'python': '3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.26100-SP0', 'event': 'saving'}
2025-08-19 19:41:53,560 : INFO : not storing attribute cum_table
2025-08-19 19:41:53,573 : INFO : saved models/yearly_models/vagabond_2013.model
2025-08-19 19:41:53,574 : INFO : Word2Vec lifecycle event {'fname_or_handle': 'models/yearly_models/vagabond_2014.model', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2025-08-19T19:41:53.574725', 'gensim': '4.3.3', 'python': '3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.26100-SP0', 'event': 'saving'}
2025-08-19 19:41:53,574 : INFO : not storing 


=== Comment Counts by Year ===
2013: 94 comments
2014: 614 comments
2015: 21598 comments
2016: 17017 comments
2017: 13738 comments
2018: 33522 comments
2019: 58153 comments
2020: 51570 comments
2021: 62415 comments
2022: 59095 comments
2023: 86296 comments
Skipping final 0 comments for 2011 (less than minimum required)
Skipping final 0 comments for 2012 (less than minimum required)
Skipping final 94 comments for 2013 (less than minimum required)
Skipping final 614 comments for 2014 (less than minimum required)
Skipping final 21598 comments for 2015 (less than minimum required)
Skipping final 17017 comments for 2016 (less than minimum required)
Skipping final 13738 comments for 2017 (less than minimum required)
Skipping final 33522 comments for 2018 (less than minimum required)
Skipping final 58153 comments for 2019 (less than minimum required)
Skipping final 51570 comments for 2020 (less than minimum required)
Skipping final 62415 comments for 2021 (less than minimum required)
Skippin

2025-08-19 19:41:53,784 : INFO : saved models/yearly_models/vagabond_2020.model
2025-08-19 19:41:53,785 : INFO : Word2Vec lifecycle event {'fname_or_handle': 'models/yearly_models/vagabond_2021.model', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2025-08-19T19:41:53.785387', 'gensim': '4.3.3', 'python': '3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.26100-SP0', 'event': 'saving'}
2025-08-19 19:41:53,785 : INFO : not storing attribute cum_table
2025-08-19 19:41:53,830 : INFO : saved models/yearly_models/vagabond_2021.model
2025-08-19 19:41:53,831 : INFO : Word2Vec lifecycle event {'fname_or_handle': 'models/yearly_models/vagabond_2022.model', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2025-08-19T19:41:53.831317', 'gensim': '4.3.3', 'python': '3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]', 'platform': 'Windows-10-1

Model saved to models/yearly_models/vagabond_2020.model
Model saved to models/yearly_models/vagabond_2021.model
Model saved to models/yearly_models/vagabond_2022.model
Model saved to models/yearly_models/vagabond_2023.model
Completed building yearly models for vagabond
